In [13]:
import pandas as pd 
import numpy as np 
from functools import reduce
import os 
import openpyxl

In [14]:
list_control = [27,29,31] 
list_shams = [5,9,11,19,21,23]
list_stimulated = [13,15,17]
rat_category = {'controls': list_control,'shams': list_shams,'stimulated': list_stimulated}

In [15]:
print(rat_category)

{'controls': [27, 29, 31], 'shams': [5, 9, 11, 19, 21, 23], 'stimulated': [13, 15, 17]}


In [16]:
'''
df_1 = dataframe with only the value of total intake, meal size etc 
df_2 = dataframe with value of total intake, meal size etc + column for rat type too 

'''

def tabulate(filenames, result_page , rat_category):
    day_wise_result = []
    final_df = pd.DataFrame()
    #for each day in filenames 
    for day in filenames: 
        if not day.endswith('.xlsx'):
            day = day + '.xlsx'
        
        df = pd.read_excel(day,sheet_name = 'PSC Totals', skiprows = 8)
        
        if result_page =='total intake':
            df_1 = df.iloc[:,7]
            
        elif result_page == 'meal number':
            df_1 = df.iloc[:,8]
            
        elif result_page == 'meal size':
            df_1 = df.iloc[:,12]
            
        elif result_page == 'intermeal interval':
            df_1 = df.iloc[:,11]
        else: 
            print("invalid result page")
            
        
    #making a list of type of rats and convert to dataframe 
        type_of_rat = [] 
        for index, row in df.iterrows():
            if row[0] in rat_category['controls']:
                type_of_rat.append('controls')
                
            elif row[0] in rat_category['shams']:
                type_of_rat.append('shams')
                
            elif row[0] in rat_category['stimulated']:
                type_of_rat.append('stimulated')
        
        type_of_rat = pd.DataFrame(np.array(type_of_rat))
        type_of_rat.columns = ['type_of_rat']
        
        df_2 = pd.concat([df_1, type_of_rat ], ignore_index=False, axis = 1)
        #now group df2 by type 
        df_2 = df_2.sort_values(by = "type_of_rat")
        #print(df_2)
        
        day_wise_result.append(df_2)
    
    
    #merging all the days dataframes into a single one    
    #result = reduce(lambda x, y: pd.merge(x, y, on = 'type_of_rat'), day_wise_result)
    for dfs in day_wise_result: 
        final_df = pd.concat([final_df,dfs.iloc[:,0]], axis = 1 )
    final_df.columns = filenames   
    final_df = pd.concat([final_df,type_of_rat.sort_values(by = 'type_of_rat')], axis = 1 )
    return final_df
    
    

In [36]:
def transfer_data(filenames, result_page_list , rat_category, output_filename):
    writer = pd.ExcelWriter(output_filename + '.xlsx', engine='xlsxwriter')
    
    final_df_list = []
    averages_list = []
    avg_shams_controls = []
    avg_stimulated_controls = []

    
    for sheetname in result_page_list: 
        if not output_filename.endswith('.xlsx'):
            output_filename = output_filename + '.xlsx'
            
        #using the tabulate function to obtain the final dataframe from the requested files    
        final_df = tabulate(filenames, sheetname, rat_category)
        
        #making dataframe containing category wise matrices 
        averages = final_df.groupby(['type_of_rat']).mean()
        
        #adding a column containing row-wise means 
        final_df['Mean'] = final_df.mean(axis=1)
        averages['Mean'] = averages.mean(axis=1)
        averages['Sum of Averages']=averages.iloc[:,0:averages.shape[1]-1].sum(axis=1)
        
        #rearranging the columns of final dataframe
        new_col_names = []
        for col in final_df.columns:
            if col != 'type_of_rat':
                new_col_names.append(col)
        new_col_names.append('type_of_rat')
        final_df = final_df[new_col_names]
        print(sheetname + " transferred")
        
        final_df_list.append(final_df)
        averages_list.append(averages)
        
        #writing the dataframes to the excel sheet 
        final_df.to_excel(writer, sheet_name=sheetname, index=False)
       
        
        #calcuating differences for each excel sheet 
        averages['diff bw shams and controls'] = averages.loc['shams'] - averages.loc['controls'] 
        averages['diff bw stimulated and controls'] = averages.loc['stimulated'] - averages.loc['controls']
        
        averages['Mean'] = averages.mean(axis=1)
        avgerages['Sum'] = averages.sum(axis=1)
        
        averages.to_excel(writer, sheet_name=sheetname, index=False, startrow=(final_df.shape[0]) + 2 , startcol= 0)
        
    writer.save()
    print("Result saved in file: " + output_filename)
    return final_df_list, averages_list


In [37]:
#final function 

#taking in inputs 
filenames = [str(x) for x in input("Enter the name of files with space in between : ").split()]
list_control = [int(x) for x in input("Enter the CONTROL RAT NO.S : ").split()]
list_shams = [int(x) for x in input("Enter the SHAM RAT NO.S : ").split()]
list_stimulated = [int(x) for x in input("Enter the STIMULATED RAT NO.S : ").split()]

#storing rat categories as signauture 
rat_category = {'controls': list_control,'shams': list_shams,'stimulated': list_stimulated}

output_filename = input("Enter the ouput filename: ")

final_df , averages = transfer_data(filenames = filenames, result_page_list=['total intake','meal number','meal size','intermeal interval'],
              rat_category = rat_category, output_filename = output_filename)

Enter the name of files with space in between : Day_1 Day_2 Day_3 Day_4
Enter the CONTROL RAT NO.S : 27 29 31
Enter the SHAM RAT NO.S : 5 9 11 19 21 23
Enter the STIMULATED RAT NO.S : 13 15 17
Enter the ouput filename: result888
total intake transferred


ValueError: No axis named 1 for object type <class 'pandas.core.series.Series'>

In [34]:
final_df

[    Day_1  Day_2  Day_3  Day_4     Mean type_of_rat
 9   16.38  14.71  20.42  16.50  17.0025    controls
 10  13.94  14.44  14.26  15.72  14.5900    controls
 11  15.86  15.24  15.87  16.22  15.7975    controls
 0   15.75  20.01  20.48  18.19  18.6075       shams
 1   17.28  20.74  16.86  21.83  19.1775       shams
 2   14.40  17.14  17.53  17.22  16.5725       shams
 6   21.33  17.14  17.72  19.29  18.8700       shams
 7   15.17  17.96  17.48  15.74  16.5875       shams
 8   18.74  19.99  20.89  17.09  19.1775       shams
 3   21.76  19.18  19.25  22.28  20.6175  stimulated
 4   19.51  18.10  17.58  16.96  18.0375  stimulated
 5   20.94  17.75  22.57  19.42  20.1700  stimulated,
     Day_1  Day_2  Day_3  Day_4  Mean type_of_rat
 9       8      6      6      6  6.50    controls
 10      6      8      5      6  6.25    controls
 11     11      9     11      8  9.75    controls
 0       6      5      5      6  5.50       shams
 1       5      7      5      7  6.00       shams
 2       7

In [35]:
# now we manually calculate some of the sums from each sheet of the resultant workbook and paste the results there 
averages


[                 Day_1      Day_2      Day_3      Day_4       Mean  \
 type_of_rat                                                          
 controls     15.393333  14.796667  16.850000  16.146667  15.796667   
 shams        17.111667  18.830000  18.493333  18.226667  18.165417   
 stimulated   20.736667  18.343333  19.800000  19.553333  19.608333   
 
              Sum of Averages  
 type_of_rat                   
 controls           63.186667  
 shams              72.661667  
 stimulated         78.433333  ,
                 Day_1     Day_2     Day_3     Day_4      Mean  Sum of Averages
 type_of_rat                                                                   
 controls     8.333333  7.666667  7.333333  6.666667  7.500000        30.000000
 shams        6.000000  7.000000  7.333333  7.166667  6.875000        27.500000
 stimulated   5.666667  6.000000  6.666667  5.000000  5.833333        23.333333,
                 Day_1     Day_2     Day_3     Day_4    Mean  Sum of Averages
 ty

In [24]:
avg_shams_controls

[Day_1    1.718333
 Day_2    4.033333
 Day_3    1.643333
 Day_4    2.080000
 dtype: float64, Day_1   -2.333333
 Day_2   -0.666667
 Day_3    0.000000
 Day_4    0.500000
 dtype: float64, Day_1    0.953333
 Day_2    0.865000
 Day_3    0.141667
 Day_4    0.280000
 dtype: float64, Day_1    438.858333
 Day_2    441.081667
 Day_3     92.113333
 Day_4    186.315000
 dtype: float64]

In [25]:
avg_stimulated_controls

[Day_1    5.343333
 Day_2    3.546667
 Day_3    2.950000
 Day_4    3.406667
 dtype: float64, Day_1   -2.666667
 Day_2   -1.666667
 Day_3   -0.666667
 Day_4   -1.666667
 dtype: float64, Day_1    1.796667
 Day_2    1.120000
 Day_3    0.443333
 Day_4    1.520000
 dtype: float64, Day_1    856.536667
 Day_2    773.423333
 Day_3    232.063333
 Day_4    601.396667
 dtype: float64]

In [26]:
sum_avg_shams_controls

[1158.3683333333333,
 1158.3683333333333,
 1158.3683333333333,
 1158.3683333333333]

In [27]:
sum_avg_stimulated_controls

[2463.42, 2463.42, 2463.42, 2463.42]